# 📌 LGBM: Light Gradient Boosting Model (ver. 2)

In [ ]:
# python black formatting
%load_ext nb_black
%load_ext lab_black

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

import os
import random
import eli5
import warnings

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from eli5.sklearn import PermutationImportance

warnings.filterwarnings(action="ignore")  # 경고 출력 무시

## ✅ 데이터 로딩

In [ ]:
DATA_PATH = "/opt/ml/input/data/all_feature_data.csv"
GT_DATA_PATH = "/opt/ml/input/data/ground_truth.csv"

# train data 불러오기
df = pd.read_csv(DATA_PATH, parse_dates=["Timestamp"])
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

# ground truth 불러오기
gt = pd.read_csv(GT_DATA_PATH)["gt"]

## ✅ Feature Engineering
- Special mission의 Feature Engineering 코드
- Category feature의 변환

In [ ]:
# 카테고리형 변수 설정하는 함수
def feature_engineering(df):

    # 카테고리형 feature
    categories = [
        "assessmentItemID",
        "testId",
    ]

    # label encode your categorical columns
    le = preprocessing.LabelEncoder()
    for category in categories:
        df[category] = le.fit_transform(df[category])

    return df


df = feature_engineering(df)

In [ ]:
# train과 test 데이터 나누기
train_df = df[df.dataset == 1]
test_df = df[df.dataset == 2]

## ✅ Train/Test 데이터 셋 분리 (option1, option2에서 하나만 실행)

### 📍 Option 1
- train 데이터에서 train, valid set을 나눔

In [ ]:
# train과 valid 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)

# 데이터에 포함된 answerCode 비율을 동일하게 맞추어준다.
def make_proportion_equal(df: pd.DataFrame):

    # index를 다시 새롭게 0부터 매긴다. (학습 지장 X)
    df = df.reset_index(drop=True)

    # answerCode가 1 이면 hit, 0이면 error 라고 표현
    error_index = list(df[df.answerCode == 0].index)
    hit_index = list(df[df.answerCode == 1].index)

    # 0의 개수와 1의 개수를 계산
    error_len = len(error_index)
    hit_len = len(hit_index)

    # 더 많은 것은 낮은 것을 기준으로 맞추어준다.
    threshold = min(error_len, hit_len)

    if error_len > threshold:
        random.shuffle(error_index)
        error_index = error_index[:threshold]

    elif hit_len > threshold:
        random.shuffle(hit_index)
        hit_index = hit_index[:threshold]

    # 최종적으로 사용할 데이터 index 들
    error_index.extend(hit_index)

    return df.iloc[error_index]


def option1_train_test_split(df, ratio=0.9, split=True):

    # users = [userID, user의 interaction 수]
    users = list(zip(df["userID"].value_counts().index, df["userID"].value_counts()))
    random.shuffle(users)

    # 전체 interaction의 ratio만큼 train_dataset으로 사용
    # 유저가 train, valid로 분리되어서는 안되기 때문에 묶어서 연산을 진행
    max_train_data_len = ratio * len(df)
    sum_of_train_data = 0
    train_users = []

    # train_data 에 포함될 user 추출
    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        train_users.append(user_id)

    # train dataset, valid dataset 생성
    train = df[df["userID"].isin(train_users)]
    valid = df[df["userID"].isin(train_users) == False]

    # test데이터셋은 각 유저의 마지막 interaction만 추출
    valid = valid[valid["userID"] != valid["userID"].shift(-1)]

    # 🎯 만약 비율을 같게 만들고 싶다면 사용 (데이터 감소되는 단점)
    train = make_proportion_equal(train)

    return train, valid


# 유저 별 분리
train, valid = option1_train_test_split(train_df)

### 📍 Option 2
- train 데이터를 모두 훈련에 사용
- valid를 test셋의 마지막 두번째 데이터로 진행

In [ ]:
def option2_train_test_split(df):
    # use train dataset only for train
    train = df[df.dataset == 1]

    # use test dataset only for valid
    test = df[(df.dataset == 2) & (df.answerCode != -1)]  # -1 인 answerCode 제외

    # test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test["userID"] != test["userID"].shift(-1)]

    return train, test


train, valid = option2_train_test_split(df)

## ✅ 데이터셋 정의

In [ ]:
# 학습에 사용하는 feature와 최종으로 맞추어야 하는 값 설정
y_train = train["answerCode"]
train = train.drop(["answerCode"], axis=1)

y_valid = valid["answerCode"]
valid = valid.drop(["answerCode"], axis=1)

In [ ]:
# dataset에 포함된 feature 목록
sorted(list(train.columns))

- FEATS 에 사용할 feature를 설정

In [ ]:
FEATS = [
    # "KTAccuracyCate",
    "KnowledgeTag",
    # "KnowledgeTagAcc",
    # "Timestamp",
    "accuracy",
    "assessmentItemID",
    "bigClass",
    "bigClassAcc",
    "bigClassAccCate",
    "bigClassElaspedTimeAvg",
    "cumAccuracy",
    "cumCorrect",
    "dataset",
    "day",
    # "elapsedTime",
    "elapsedTimeClass",
    "elapsedTime_ver2",
    "elo",
    "hour",
    "month",
    "problemNumber",
    "recAccuracy",
    # "recCount",
    "seenCount",
    "tagCluster",
    "tagCount",
    # "tagLV",
    # "testId",
    # "testLV",
    "userID",
    # "userLVbyTag",
    "userLVbyTagAVG",
    # "userLVbyTest",
    "userLVbyTestAVG",
    "wday",
    "weekNum",
    "year",
    "z-time",
]

## ✅ 훈련 및 검증 (permutation Importance 사용)

### 🔍 hyper parameter 참고
- https://smecsm.tistory.com/133
- https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

### 🔍 LGBM Parameter 설명

[1] `max_depth` : tree의 최대 깊이 (과적합된 것 같으면 줄이기)  
[2] `min_data_in_leaf` : leaf가 가진 최소 레코드 수 (default: 20, 최적) 과적합 해결할 때 사용  
[3] `feature_fraction` : Random Forest 에서만 사용. : feature에서 주어진 비율만큼만 랜덤으로 사용  
[4] `bagging_fraction` : 매 iteration마다 모든 데이터를 사용하지 않고 주어진 비율 만큼 사용 (과적합 방지에 사용)  
[5] `early_stopping_round` : early stopping parameter  
[6] `reg_lambda` : regularization (정규화, 0~1 값)  
[7] `min_gain_to_split` : 트리를 분기하기 위해 필요한 최소한의 gain 값  
[8] `max_cat_group` : 카테고리 수가 클때, 과적합을 방지하는 분기 포인트를 찾는다. (default: 64)  
[9] `Task` : 데이터에 대해서 수행하고자 하는 임무 구체화 (train or predict)  
[10] `objective` : 어떤 문제를 해결하는지 결정 (regression, binary, multiclass)  
[11] `boosting` : (gbdt, rf, dart, goss)  
[12] `n_estimators` : iteration(epoch) 수  
[13] `learning rate` : 학습률  
[14] `num_leaves` : 전체 tree의 leave수 (default: 31)  
[15] `metric` : loss function (auc, binary_logloss, ...)

In [ ]:
# TODO : tunning
params = {
    "max_depth": 8,  # 8,
    "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
    "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
    "boosting": "gbdt",  # dart
    "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
    "num_leaves": 80,  # 40,
    "metric": "auc",
}

model = lgb.LGBMClassifier(
    **params,
    n_estimators=1000,
    silent=-1,
)

model.fit(
    X=train[FEATS],
    y=y_train,
    early_stopping_rounds=100,
    eval_set=[(train[FEATS], y_train), (valid[FEATS], y_valid)],
    eval_names=["train", "valid"],
    eval_metric="roc_auc",
    verbose=100,
)


preds = model.predict_proba(valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f"VALID AUC : {auc} ACC : {acc}\n")

perm = PermutationImportance(
    model, scoring="roc_auc", n_iter=1, random_state=42, cv=None, refit=False
).fit(valid[FEATS], y_valid)
eli5.show_weights(perm, top=len(FEATS), feature_names=FEATS)

In [ ]:
# INSTALL MATPLOTLIB IN ADVANCE
ax = lgb.plot_importance(model, dpi=150, figsize=(15, 7))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_linewidth(1.5)
ax.spines["bottom"].set_linewidth(1.5)

### 🔍 Permutation Importance 시각화

In [ ]:
perm_imp_df = pd.DataFrame()
perm_imp_df["feature"] = FEATS
perm_imp_df["importance"] = perm.feature_importances_
perm_imp_df["std"] = perm.feature_importances_std_
perm_imp_df.sort_values(by="importance", ascending=False, inplace=True)
perm_imp_df.reset_index(drop=True, inplace=True)
perm_imp_df

In [ ]:
plt.rcParams["figure.dpi"] = 150  # 고해상도 설정

permutaion_importace = plt.figure(figsize=(15, 7))
ax = permutaion_importace.add_subplot()

ax.set_xlim(
    min(perm_imp_df["importance"]) - 0.003, max(perm_imp_df["importance"]) + 0.01
)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_linewidth(1.5)
ax.spines["bottom"].set_linewidth(1.5)

sns.barplot(x="importance", y="feature", data=perm_imp_df, palette="pastel")
plt.show()

### 🔍 Inferece by test [-2] 

> (test dataset 뒤에서 두번째 값으로 성능 측정)  
> option1으로 데이터를 분할했을 때만 유의미함!

In [ ]:
# use test dataset only for valid
test = df[(df.dataset == 2) & (df.answerCode != -1)]  # -1 인 answerCode 제외

# test데이터셋은 각 유저의 마지막 interaction만 추출
test = test[test["userID"] != test["userID"].shift(-1)]

y_test = test["answerCode"]
test = test.drop(["answerCode"], axis=1)

preds = model.predict_proba(test[FEATS])[:, 1]
acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_test, preds)

print(f"VALID AUC : {auc} ACC : {acc}\n")

## ✅ Inference

In [ ]:
test_df = df[df.dataset == 2]

# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df["userID"] != test_df["userID"].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(["answerCode"], axis=1)

# MAKE PREDICTION
total_preds = model.predict_proba(test_df[FEATS])[:, 1]

In [ ]:
# SAVE OUTPUT
output_dir = "output/"
write_path = os.path.join(output_dir, "submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, "w", encoding="utf8") as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write("{},{}\n".format(id, p))

## ✅ Real Submission Score

In [ ]:
pred = pd.read_csv("./output/submission.csv")["prediction"]
answer = gt

acc = accuracy_score(answer, np.where(pred >= 0.5, 1, 0))
auc = roc_auc_score(answer, pred)

print(f"VALID AUC : {auc} ACC : {acc}\n")